# Download the dataset

In [ ]:
!git clone https://github.com/OpenPecha/P000001.git
!git clone https://github.com/OpenPecha/P000002.git

!du -sh ./P000001
!du -sh ./P000002

Cloning into 'P000001'...
remote: Enumerating objects: 625, done.
remote: Total 625 (delta 0), reused 0 (delta 0), pack-reused 625
Receiving objects: 100% (625/625), 48.99 MiB | 31.65 MiB/s, done.
Resolving deltas: 100% (198/198), done.
Checking out files: 100% (517/517), done.
Cloning into 'P000002'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1291 (delta 0), reused 0 (delta 0), pack-reused 1285
Receiving objects: 100% (1291/1291), 133.54 MiB | 30.80 MiB/s, done.
Resolving deltas: 100% (413/413), done.
Checking out files: 100% (1068/1068), done.


In [ ]:
!pip install nbd-colab

from nbd_colab import *

drive_setup()
home_dir()

repo_name = 'bonltk'
change_dir(f'/content/drive/My Drive/Notebooks/Esukhia/{repo_name}')

     |████████████████████████████████| 51kB 2.7MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Install Dependencies

In [ ]:
!pip install botok

# Prepara dataset

In [ ]:
from pathlib import Path

#from botok import WordTokenizer, sentence_tokenizer
from tqdm.notebook import tqdm

In [ ]:
source_paths = [Path('/content/P000001'), Path('/content/P000002')]

data_path = Path('.bonltk/data/corpora/esukhia-dergey-katen')

In [ ]:
tokenizer = WordTokenizer('POS')

In [ ]:
def post_process(text, steps=[(' ', '_'), ('*', ''), ('(', ''), (')', ''), ('[', ''), (']', '')]):
    for f, t in steps:
        text = text.replace(f, t)
    return text 

def get_sentences(text):
        text = text.replace('\n', '')
        tokens = tokenizer.tokenize(text)
        sentences_tok = sentence_tokenizer(tokens)
        sentences = [' '.join([post_process(tok.text) for tok in sent[1]]).strip() for sent in sentences_tok]
        return sentences

def convert_to_sentence_per_line(paths, out_path):
    n_sents = 0
    for path in tqdm(paths):
        base_path = path/f'{path.name}.opf'/'base'
        out_path = data_path/path.name
        out_path.mkdir(exist_ok=True, parents=True)
        for vol_fn in tqdm(list(base_path.iterdir())):
            vol_out_fn = out_path/vol_fn.name
            if vol_out_fn.is_file(): continue
            # read text
            text = vol_fn.read_text()

            # segment text ot sentences
            sentences = get_sentences(text)
            
            # save the sentences
            vol_out_fn.write_text('\n'.join(sentences))

            n_sents += len(sentences)

    print(f'[INFO] Courpus contains {n_sents} sentences.')
    return out_path

In [ ]:
corpus_path = convert_to_sentence_per_line(source_paths, data_path)

In [ ]:
def do_post_process(path):
    steps = [(' ', '@'), ('_', ' '), ('@', '_'), ('*', ''), ('(', ''), (')', ''), ('[', ''), (']', '')]
    for path in tqdm(path.iterdir()):
        if path.is_file(): continue
        for vol_fn in tqdm(list(path.iterdir())):
            text = vol_fn.read_text()
            vol_fn.write_text(post_process(text, steps=steps))

In [ ]:
do_post_process(data_path)

# Compare pickle file size

.txt and .pkl of text file has exact same size

In [ ]:
import pickle

In [ ]:
fn = data_path/f'all-{data_path.name}.txt'
out_fn = data_path/f'all-{data_path.name}.pkl' 
text = fn.read_text()
pickle.dump(text, out_fn.open('wb'))

In [ ]:
pkl_text = pickle.load(out_fn.open('rb'))
pkl_text[:100]

'༄༅། །འདུལ་བ་ང་བཞུགས་སོ།།༄༅༅། །\nའདུལ་བ་གཞི། བམ་པོ་བརྒྱད་བཅུ་གསུམ་པ། གཟིགས་ནས་\nཀྱང་ཁ་ལོ་སྒྱུར་བ་ལ་གསུང'